In [2]:
import gensim
from gensim import corpora
import torch
from transformers import BertForTokenClassification, BertTokenizer
import gensim
from gensim import corpora
import nltk
from nltk.corpus import stopwords
import torch
from transformers import BertForTokenClassification, BertTokenizer

# 下载 NLTK 数据
nltk.download('punkt')
nltk.download('stopwords')
def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    return tokens

# 主题模型关键词抽取
def get_topic_keywords(corpus, num_topics=5, num_keywords=10):
    # 创建字典
    dictionary = corpora.Dictionary(corpus)
    
    # 创建语料库
    corpus = [dictionary.doc2bow(text) for text in corpus]
    
    # 训练 LDA 模型
    ldamodel = gensim.models.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary)
    
    # 获取每个主题的关键词
    topic_keywords = [[(dictionary[id], prob) for id, prob in ldamodel.get_topic_terms(topicid, topn=num_keywords)]
                      for topicid in range(ldamodel.num_topics)]
    
    return topic_keywords

# BERT 关键词抽取
def get_bert_keywords(texts, model, tokenizer, max_len=512):
    all_keywords = []
    for text in texts:
        # 对文本进行编码
        encoding = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_len, 
                                         return_tensors='pt', truncation=True)
        
        # 获取输出
        output = model(encoding['input_ids'], encoding['attention_mask'])[0]
        
        # 获取关键词索引
        keyword_indices = torch.where(output[0] > 0)[0].tolist()
        
        # 解码关键词
        keywords = tokenizer.convert_ids_to_tokens([encoding['input_ids'][0][i] for i in keyword_indices])
        
        all_keywords.append(keywords)
        
    return all_keywords

# 示例用法
corpus = [
    "This is a sample document about Python programming language.",
    "Python is a high-level, general-purpose programming language.",
    "It was created by Guido van Rossum in the late 1980s."
]

# 对文本进行预处理
preprocessed_corpus = [preprocess(text) for text in corpus]

# 主题模型关键词
topic_keywords = get_topic_keywords(preprocessed_corpus, num_topics=2, num_keywords=5)
print("Topic Model Keywords:")
for topic, keywords in enumerate(topic_keywords):
    print(f"Topic {topic}: {', '.join([word for word, prob in keywords])}")

# 主题模型关键词
topic_keywords = get_topic_keywords(corpus, num_topics=2, num_keywords=5)
print("Topic Model Keywords:")
for topic, keywords in enumerate(topic_keywords):
    print(f"Topic {topic}: {', '.join([word for word, prob in keywords])}")

# BERT 关键词抽取
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')
bert_keywords = get_bert_keywords(corpus, model, tokenizer)
print("\nBERT Keywords:")
for text, keywords in zip(corpus, bert_keywords):
    print(f"Text: {text}")
    print(f"Keywords: {', '.join(keywords)}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\18353\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\18353\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Topic Model Keywords:
Topic 0: ., language, python, programming, guido
Topic 1: ., programming, python, language, ,


TypeError: doc2bow expects an array of unicode tokens on input, not a single string